In [ ]:
# evaluation.ipynb

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib 
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. 모델 및 인코더 로드
try:
    # training.ipynb에서 저장했던 모델 파일과 인코더 파일을 불러옴
    model = joblib.load('pitcher_model.pkl')
    le = joblib.load('pitch_type_encoder.pkl')
    print("모델 및 인코더 로드 성공!")
except FileNotFoundError:
    print("오류: 모델 파일(pitcher_model.pkl) 또는 인코더 파일(pitch_type_encoder.pkl)을 찾을 수 없습니다.")
    exit()

# 2. 테스트 데이터 준비 
file_path = '../assignment4/data/pitcher_data.csv'
df = pd.read_csv(file_path)

features = ['balls', 'strikes', 'on_1b', 'on_2b', 'on_3b', 'outs_when_up', 'inning']
target = 'pitch_type'

# 전처리
df = df.dropna(subset=[target])
X = df[features]
y = df[target]

X = X.fillna(0) 

# y 인코딩
y_encoded_full = le.transform(y) 

# 3. 데이터 분할 (Split - random_state=42 유지)
# 테스트 데이터셋을 다시 분할합니다.
X_train_val, X_test, y_train_val_encoded, y_test_encoded = train_test_split(X, y_encoded_full, test_size=0.2, random_state=42)
# X_test, y_test_encoded는 최종 평가 데이터입니다.


# 4. 모델 성능 평가 (Evaluate)
print("\n=== 모델 성능 평가 시작 ===")

# 테스트 데이터로 예측 수행
y_pred_encoded = model.predict(X_test)

# 5. 주요 평가 지표 계산 및 출력
# 다중 클래스 분류이므로 average='macro'를 사용하여 모든 클래스를 동일하게 평가.
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
precision_macro = precision_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0)
recall_macro = recall_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0)
f1_macro = f1_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0)


# 6. 결과 출력
print(f"총 테스트 데이터 수: {len(X_test)}개")
print(f"정확도 (Accuracy): {accuracy:.4f}")
print(f"정밀도 (Precision, 평균): {precision_macro:.4f}")
print(f"재현율 (Recall, 평균): {recall_macro:.4f}")
print(f"F1-Score (F1 점수): {f1_macro:.4f}")


# 7. 혼동 행렬 (Confusion Matrix) 출력 및 시각화

# 7-1. 혼동 행렬 계산
# labels 인수에 le.transform(le.classes_)를 넣어 5x5 행렬이 되도록 강제
# le.classes_는 학습 때 사용된 구종 이름 목록
all_classes = le.transform(le.classes_)
cm = confusion_matrix(y_test_encoded, y_pred_encoded, labels=all_classes)
pitch_names = le.classes_
cm_df = pd.DataFrame(cm, index=pitch_names, columns=pitch_names)

print("\n=== 혼동 행렬 (Confusion Matrix) ===")
print("  (행: 실제 정답, 열: 모델 예측값)")
display(cm_df)

# 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix: Pitch Type Prediction')
plt.ylabel('실제 구종 (True Pitch Type)')
plt.xlabel('예측된 구종 (Predicted Pitch Type)')
plt.show()